In [2]:
import os
from langame.langame_client import LangameClient
import random
import openai
from firebase_admin import firestore

import glob

c = LangameClient(path_to_config_file="../config.yaml")

In [4]:
memes = []
for e in c._firestore_client.collection('memes').stream():
    memes.append((e.id, e.to_dict()))

In [5]:
len(memes)

707

In [6]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
def get_prompt(topics):
    rnd = random.choices(memes, k=500)
    return [(e[1]['content'], e[1]['topics']) for e in rnd if len(intersection(e[1]['topics'], topics)) > 0]

In [7]:
set([",".join(e[1]['topics']) for e in memes])

{'anthropology,philosophy',
 'art',
 'art,book,film',
 'art,ice breaker',
 'art,religion',
 'artificial intelligence',
 'artificial intelligence,art,philosophy',
 'artificial intelligence,biology',
 'artificial intelligence,knowledge',
 'artificial intelligence,philosophy',
 'artificial intelligence,philosophy,knowledge',
 'artificial intelligence,robotic',
 'artificial intelligence,transhumanism',
 'artificial intelligence,transhumanism,robotic',
 'biology',
 'biology,artificial intelligence',
 'biology,business,startup',
 'biology,mind',
 'biology,philosophy',
 'biology,politic,transhumanism,philosophy,artificial intelligence',
 'biology,religion',
 'biotechnology',
 'book',
 'book,ice breaker',
 'book,ice breaker,mind',
 'business',
 'business,religion',
 'business,startup',
 'ecology',
 'ecology,artificial intelligence,philosophy',
 'ecology,biology',
 'ecology,business',
 'ecology,health',
 'ecology,philosophy',
 'ecology,science-fiction',
 'ecology,technology',
 'economic,philoso

In [8]:
from tqdm import tqdm
import time
from random import randint

outputs = []
for i in tqdm(range(10**12)):
  time.sleep(randint(1,3))
  samples = get_prompt(['science', 'psychology'])
  p = list(str(samples[0:60]))
  # replace last char by a ","
  p[-1] = ","
  p = "".join(p)
  try:
    response = openai.Completion.create(
      engine="davinci-codex",
      prompt=p,
      temperature=1,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.5,
      presence_penalty=0,
      stop=["])"]
    )
  except Exception as e:
    print(e)
    continue
  text = response["choices"][0]["text"] + "]),"
  outputs.append(text)
  # every 10 samples, append the generated samples to file
  if i % 10 == 0:
    with open("generated_samples.txt", "a") as f:
      f.write("\n".join(outputs) + "\n")
    outputs = []

  0%|          | 1236/1000000000000 [1:09:53<779084700:27:08,  2.80s/it]

This model's maximum context length is 4097 tokens, however you requested 4316 tokens (4216 in your prompt, 100 for the completion). Please reduce your prompt or completion length.


  0%|          | 2470/1000000000000 [2:20:14<1026148943:33:08,  3.69s/it]

In [9]:
def is_valid():
    invalids = []
    for e in n_m:
        if not isinstance(e[0], str) or not isinstance(e[1], list):
            invalids.append(e)
    return invalids

In [10]:
is_valid()

[]

In [51]:
for e in n_m:
    d = {
        'content': e[0], 
        'topics': e[1],
        'disabled': False,
        'createdAt': firestore.SERVER_TIMESTAMP,
        'tweet': True,
    }
    c._firestore_client.collection('memes').add(d)

In [36]:
import re
from glob import glob
from transformers import T5Tokenizer, T5ForConditionalGeneration
from langame.strings import string_similarity
import torch
import json
model_name = "flexudy/t5-small-wav2vec2-grammar-fixer"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = T5Tokenizer.from_pretrained(model_name)

model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
regex = re.compile(r"[^a-zA-Z0-9,?]")
nd = []
# regex that include all characters usually in conversational text
# ('Which is the main purpose of artificial intelligence in your opinion, what will it help us to achieve?', ['artificial intelligence']),
for f in glob("../generated_samples.txt"):
        with open(f, "r") as f:
            splitted = f.read().split("\n")
            for s in splitted:
                try:
                    parsed = json.loads(s)
                    if "content" not in parsed:
                        continue
                    input_text = "fix: { " + parsed["content"] + " } </s>"
                    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True, add_special_tokens=True).to(device)

                    outputs = model.generate(
                        input_ids=input_ids,
                        max_length=256,
                        num_beams=4,
                        repetition_penalty=1.0,
                        length_penalty=1.0,
                        early_stopping=True
                    )

                    sentence = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
                    
                    if len(sentence) > 20 and string_similarity(parsed["content"], sentence) > 0.5:
                        print("original:", parsed["content"], "fixed:", sentence)

                        nd.append(sentence)
                except Exception as exc:
                    # print(exc)
                    continue



original: If you were so rich you didn't need to work and you don’t have to worry about money all the time, what would you do with your time? fixed: If you were so rich you didn’t need to work and you don’t have to worry about money all the time, what would you do with your time?
original: "What do you envy about your friend's life? What do you wish was in your life that is not? fixed: What do you envy about your friend's life? What do you wish was in your life that is not?
original: Do you feel as though your dreams are realistic or far fetched? fixed: Do you feel as though your dreams are realistic or far faretched?
original: If you were an evil person, what would be a reason you would want to torture someone? fixed: If you were an evil person, what would be a reason you would want to torture someone?
original: If you could live forever, would you? fixed: If you could live forever, would you?
original: If you could buy a lacquer, what brand of nail polish would it be? fixed: If you c

In [35]:
# write nd to file
with open("../data/augment/all3.txt", "w") as f:
    f.write("\n".join(nd))